# Ionisation potential of a porous material

In this example we use `MacroDensity` with `VASP` to align the energy levels of a porous material.

The procedure involves one DFT calculaion, yielding different important values

* $\epsilon_{vbm}$ - the valence band maximum
* $V_{vac}$ - the vacuum potential

The ionisation potential ($IP$) is then obtained from:

$IP = V_{vac} - \epsilon_{vbm}$

The difference to a bulk calculation is that here the material itself has a vacuum within. That means that we can sample the vacuum level from there.

The procedure was first outlined in a seminal JACS paper, [read it here](http://pubs.acs.org/doi/abs/10.1021/ja4110073).

## Our system 

The beautiful ZIF-8 is our porous system of choice for this demonstration.

<img src="zif8.png">

## Procedure

* Optimise the structre
* Calculate the electronic structure at your chosen level of theory **Remember in your INCAR:**

    LVHAR = .TRUE.  # This generates a LOCPOT file with the potential 
    
* Locate the centre of the largest pore - do this "by eye" first
* Plot the potential in that plane, so see if it plateaus
* Plot a profile of the potential across the pore, again to see the plateau
* Sample the potential from the pore centre

**NB** This whole procedure is probably better run in a notebook than by script, the reason being that you can read the file once, then do the manipulations later. The reading is the intensive and time consuming step. 

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import imp
import NewPotentialModule as pot
import math
import numpy as np
import matplotlib.pyplot as plt

### Read the potential

In [30]:
input_file = 'AFI/LOCPOT'
#=== No need to edit below
vasp_pot, NGX, NGY, NGZ, Lattice = pot.read_vasp_density(input_file)
vector_a,vector_b,vector_c,av,bv,cv = pot.matrix_2_abc(Lattice)
resolution_x = vector_a/NGX
resolution_y = vector_b/NGY
resolution_z = vector_c/NGZ
grid_pot, electrons = pot.density_2_grid(vasp_pot,NGX,NGY,NGZ)

Reading potential at point 100000
Reading potential at point 200000
Reading potential at point 300000
Reading potential at point 400000
Reading potential at point 500000
Reading potential at point 600000
Reading potential at point 700000
Reading potential at point 800000
Reading potential at point 900000
Reading potential at point 1000000
Reading potential at point 1100000
Reading potential at point 1200000
Reading potential at point 1300000
Reading potential at point 1400000
Reading potential at point 1500000
Reading potential at point 1600000
Reading potential at point 1700000
Reading potential at point 1800000
Reading potential at point 1900000
Reading potential at point 2000000
Reading potential at point 2100000
Reading potential at point 2200000
Reading potential at point 2300000
Reading potential at point 2400000
Reading potential at point 2500000
Reading potential at point 2600000
Reading potential at point 2700000
Reading potential at point 2800000
Reading potential at point 29

In [31]:
cube_size = [4,4,4]
cube_origin = [0,0,0.5]

In [32]:
## cube defines the size of the cube in units of mesh points (NGX/Y/Z)
cube = cube_size
## origin defines the bottom left point of the cube the "0,0,0" point in fractional coordinates
origin = cube_origin
## travelled; do not alter this variable
travelled = [0,0,0]
## Uncomment the lines below to do the business
cube_potential, cube_var = pot.cube_potential(origin,travelled,cube,grid_pot,NGX,NGY,NGZ)
print "Potential            Variance"
print "--------------------------------"
print cube_potential,"   ", cube_var

Potential            Variance
--------------------------------
5.80630187979     2.23093986327e-08


In [18]:
%%bash
nkpts=`grep NKPTS AFI/OUTCAR | awk '{print $4}'`
uband=`grep NELECT AFI/OUTCAR | awk '{print $3/2}'`
grep "  $uband   " AFI/OUTCAR | tail -$nkpts | awk '{print $2}' | sort -n | head | tail -1

-2.0729


In [19]:
print "IP: %3.4f eV" % (5.80630187979 -- 2.0729 )

IP: 7.8792 eV


## LTA

In [20]:
input_file = 'LTA/LOCPOT'
#=== No need to edit below
vasp_pot, NGX, NGY, NGZ, Lattice = pot.read_vasp_density(input_file)
vector_a,vector_b,vector_c,av,bv,cv = pot.matrix_2_abc(Lattice)
resolution_x = vector_a/NGX
resolution_y = vector_b/NGY
resolution_z = vector_c/NGZ
grid_pot, electrons = pot.density_2_grid(vasp_pot,NGX,NGY,NGZ)

Reading potential at point 100000
Reading potential at point 200000
Reading potential at point 300000
Reading potential at point 400000
Reading potential at point 500000
Reading potential at point 600000
Reading potential at point 700000
Reading potential at point 800000
Reading potential at point 900000
Reading potential at point 1000000
Reading potential at point 1100000
Reading potential at point 1200000
Reading potential at point 1300000
Reading potential at point 1400000
Reading potential at point 1500000
Reading potential at point 1600000
Reading potential at point 1700000
Reading potential at point 1800000
Reading potential at point 1900000
Reading potential at point 2000000
Reading potential at point 2100000
Reading potential at point 2200000
Reading potential at point 2300000
Reading potential at point 2400000
Reading potential at point 2500000
Reading potential at point 2600000
Reading potential at point 2700000
Reading potential at point 2800000
Reading potential at point 29

In [22]:
cube_size = [4,4,4]
cube_origin = [0,0,0]
## cube defines the size of the cube in units of mesh points (NGX/Y/Z)
cube = cube_size
## origin defines the bottom left point of the cube the "0,0,0" point in fractional coordinates
origin = cube_origin
## travelled; do not alter this variable
travelled = [0,0,0]
## Uncomment the lines below to do the business
cube_potential, cube_var = pot.cube_potential(origin,travelled,cube,grid_pot,NGX,NGY,NGZ)
print "Potential            Variance"
print "--------------------------------"
print cube_potential,"   ", cube_var

Potential            Variance
--------------------------------
4.47729686722     4.32406831814e-08


In [24]:
%%bash
nkpts=`grep NKPTS LTA/OUTCAR | awk '{print $4}'`
uband=`grep NELECT LTA/OUTCAR | awk '{print $3/2}'`
grep "  $uband   " LTA/OUTCAR | tail -$nkpts | awk '{print $2}' | sort -n | head | tail -1

-3.0078


In [25]:
print "IP: %3.4f eV" % (4.47729686722 -- 3.0078 )

IP: 7.4851 eV


## SOD

In [33]:
input_file = 'SOD/LOCPOT'
#=== No need to edit below
vasp_pot, NGX, NGY, NGZ, Lattice = pot.read_vasp_density(input_file)
vector_a,vector_b,vector_c,av,bv,cv = pot.matrix_2_abc(Lattice)
resolution_x = vector_a/NGX
resolution_y = vector_b/NGY
resolution_z = vector_c/NGZ
grid_pot, electrons = pot.density_2_grid(vasp_pot,NGX,NGY,NGZ)

Reading potential at point 100000
Reading potential at point 200000
Reading potential at point 300000
Reading potential at point 400000
Reading potential at point 500000
Reading potential at point 600000
Reading potential at point 700000
Reading potential at point 800000
Reading potential at point 900000
Reading potential at point 1000000
Reading potential at point 1100000
Reading potential at point 1200000
Reading potential at point 1300000
Reading potential at point 1400000
Reading potential at point 1500000
Reading potential at point 1600000
Reading potential at point 1700000
Reading potential at point 1800000
Reading potential at point 1900000
Reading potential at point 2000000
Reading potential at point 2100000
Reading potential at point 2200000
Reading potential at point 2300000
Reading potential at point 2400000
Reading potential at point 2500000
Reading potential at point 2600000
Reading potential at point 2700000
Reading potential at point 2800000
Reading potential at point 29

In [34]:
cube_size = [4,4,4]
cube_origin = [0,0,0]
## cube defines the size of the cube in units of mesh points (NGX/Y/Z)
cube = cube_size
## origin defines the bottom left point of the cube the "0,0,0" point in fractional coordinates
origin = cube_origin
## travelled; do not alter this variable
travelled = [0,0,0]
## Uncomment the lines below to do the business
cube_potential, cube_var = pot.cube_potential(origin,travelled,cube,grid_pot,NGX,NGY,NGZ)
print "Potential            Variance"
print "--------------------------------"
print cube_potential,"   ", cube_var

Potential            Variance
--------------------------------
5.36814762016     1.44905283289e-07


In [35]:
%%bash
nkpts=`grep NKPTS SOD/OUTCAR | awk '{print $4}'`
uband=`grep NELECT SOD/OUTCAR | awk '{print $3/2}'`
grep "  $uband   " SOD/OUTCAR | tail -$nkpts | awk '{print $2}' | sort -n | head | tail -1

-1.9688


In [36]:
print "IP: %3.4f eV" % (5.36814762016 -- 1.9688 )

IP: 7.3369 eV
